$$ \frac{\partial C}{\partial Z}= CU , \frac{\partial E}{\partial Z} =0, \frac{\partial S}{\partial Z} =0$$
$$FC=SCE $$
$$\frac{\partial F}{\partial Z}C + FCU =SCUE $$

$$C^\dagger \frac{\partial F}{\partial Z_\alpha} C =C^\dagger SCUE - C^\dagger FCU $$
$$C^\dagger \frac{\partial F}{\partial Z_\alpha} C = UE -EU $$

Prima approsimazione stupida è di fare dF==dV e scrivere
$$ C^\dagger \frac{\partial F}{\partial Z_\alpha} C =C^\dagger \frac{\partial V_{ne}}{\partial Z_\alpha} C = UE - EU $$
E poi posso anche scomporre:
$$ \left( UE - EU \right)_{ij} = \sum_k U_{ik}\delta_{kj} \epsilon_j - \epsilon_i \delta_{ik} U_{kj} = U_{ij}(\epsilon_j - \epsilon_i)$$
e quindi:
$$ U_{ij} \approx \left ( C^\dagger \frac{\partial V_{ne}}{\partial Z_\alpha} C \right)_{ij} / (\epsilon_j - \epsilon_i) $$
ove $(\epsilon_j - \epsilon_i) \ne 0 $

For the derivatives of the density matrix, using the skew symmetry of the response matrix $U=-U^\dagger$:
$$ P=COC^\dagger$$
$$\frac{\partial P}{\partial Z_\alpha}=CUOC^\dagger + COU^\dagger C^\dagger = C(UO-OU)C^\dagger $$

Then use this expression to solve iteratively:
$$ C^\dagger \frac{\partial F}{\partial Z_\alpha} C =C^\dagger  \frac{\partial \left( V_{ne}+ V_{ee} \right ) }{\partial Z_\alpha} C = UE - EU $$
Where:
$$ \frac{\partial V_{ee}}{\partial Z_\alpha} = \sum_{kl}\left( g_{ikjl} - g_{ijkl}/2 \right ) \left( \frac{\partial P}{\partial Z_\alpha} \right ) _{kl} $$
And if $E$ is diagonal:
$$ \left( UE -EU   \right)_{ij}= \sum_k\left( U_{ik}E_{kj} -E_{ik}U_{kj} \right)= U_{ij}\left( \epsilon_j -\epsilon_i  \right) $$

$$ C^\dagger  \left(dV+ \sum_{kl}(G_{ikjl} dP_{kl}) \right)C = U_{ij}\left( \epsilon_j -\epsilon_i  \right) $$
$$ dP= C(UO-OU)C^\dagger $$

In [68]:
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
# ancora il modo migliore per mettere cariche frazionali sulle molecole, se vai a toccare mol.charges si incazza 
#  quando deve calcolare l' initial guess 
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords()/angstrom, deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [69]:
co=gto.Mole(atom="C 0 0 0 ;O 0 0 2.1", basis= "cc-pvtz",units="Bohr",maxit=100)
hfco=scf.RHF(co)
hfco.scf()

Initialize <pyscf.gto.mole.Mole object at 0x7f0311ecccd0> in <pyscf.scf.hf.RHF object at 0x7f0317247310>


converged SCF energy = -112.344165614949


-112.34416561494943

In [70]:
dL=.0001
hf1=fc(hfco,[dL,-dL])
hf1.scf()
hf2=fc(hfco,[-dL,dL])
hf2.scf()
dV=DeltaV(co,[dL,-dL])
print(np.allclose(dV,hf1.get_hcore()-hfco.get_hcore()))

converged SCF energy = -112.343388907244
converged SCF energy = -112.344942384869
True


In [71]:
C=hfco.mo_coeff
S=hfco.get_ovlp()
e=hfco.mo_energy
E=np.diag(e)
O=np.diag(hfco.mo_occ)
C1=hf1.mo_coeff
P=hfco.make_rdm1()
P1=hf1.make_rdm1()
P2=hf2.make_rdm1()
dP=(P1-P2)/2
dC=(abs(C1)*C/abs(C)-C)

/home/giorgiod/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [72]:
h2=hfco.get_veff()
J,K=hfco.get_jk()
g_ijkl=hfco.mol.intor('int2e_sph')
print( "J-K/2 = H2 :",np.allclose(h2,J-K/2))

J-K/2 = H2 : True


In [73]:
print(np.allclose(np.einsum('ijkl,kl->ij',g_ijkl,P ),J))
print(np.allclose(np.einsum('ijkl,jl->ik',g_ijkl,P ),K))

True
True


In [74]:
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2
print(np.allclose(np.einsum('ijkl,kl->ij',g_h2,P ),h2))


True


In [75]:
U_app=C.T.dot(dV.dot(C))  # approximate value of U
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            U_app[i][j]/=(e[j]-e[i])
dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T)
#plt.matshow(dP_app)
#plt.matshow(dP)
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))
print(np.linalg.norm(dP_app-dP))

0.00026285014957900106
0.0003110698009103365
0.00010017252890256193


In [80]:
TM= (U_app.dot(O)-O.dot(U_app))
np.allclose(TM,TM.T)

True

In [66]:
for i in range(10):
    U_app=C.T.dot((dV+np.einsum('ijkl,kl->ij',g_h2,dP_app)).dot(C)) # approximate value of U
    for i in range(co.nao):
        for j in range(co.nao):
            if e[i]!=e[j]:
                U_app[i][j]/=(e[j]-e[i])
    dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T)
    #plt.matshow(dP_app)
    #plt.matshow(dP)
    print(np.linalg.norm(dP))
    print(np.linalg.norm(dP_app))
    print(np.linalg.norm(dP_app-dP))

0.00026285014958914664
0.00026278924131348545
1.589157256454941e-07
0.00026285014958914664
0.0002628989832604594
1.1212802049821637e-07
0.00026285014958914664
0.00026281842150069503
9.255032749652943e-08
0.00026285014958914664
0.0002628776499222679
6.57851838004388e-08
0.00026285014958914664
0.00026283416263212585
5.969920732378586e-08
0.00026285014958914664
0.00026286612900117715
4.511099729641424e-08
0.00026285014958914664
0.00026284265487781387
4.50601460401495e-08
0.00026285014958914664
0.00026285990781325794
3.775953698072597e-08
0.00026285014958914664
0.00026284723683841024
3.926848105654408e-08
0.00026285014958914664
0.00026285654874734786
3.5774955670255216e-08


In [67]:
dEnn=(49-48)/2.1
print("true target energy: ",fc(hfco,[1,-1]).scf())
print("CO energy = ",hfco.e_tot)
print("APDFT1 energy =", hfco.e_tot+dEnn+np.trace(P.dot(dV/dL)))
print("APDFT2 energy =", hfco.e_tot+dEnn+np.trace((P+0.5*dP/dL).dot(dV/dL)))
print("APDFT2 energy approx.=", hfco.e_tot+dEnn+np.trace((P+0.5*dP_app/dL).dot(dV/dL)))

converged SCF energy = -107.659464415009
true target energy:  -107.65946441500944
CO energy =  -112.34416561494926
APDFT1 energy = -104.6045652752867
APDFT2 energy = -107.46337612914532
APDFT2 energy approx.= -107.4634327704485


In [80]:
np.trace((P+dP_app).dot(S))

13.999999999999993

#### Comments
It works pretty well for minimal basis set, good results were obtained with few iterations. <br>
For larger basis set an acceptable convergence can be obtained only after 20-30 iterations <br>
dL affects the results for dP calculated numerically, but not analytically, thus is to be chosen a small dL ($10^{-3} / 10^{-4} $) in order to compare dP_approximated with a proper numerical dP.
